In [1]:
!pip install gensim

     |████████████████████████████████| 24.2 MB 9.4 MB/s eta 0:00:01     |██████████████████████          | 16.7 MB 8.9 MB/s eta 0:00:01
     |████████████████████████████████| 113 kB 6.2 MB/s eta 0:00:01
  Created wheel for smart-open: filename=smart_open-3.0.0-py3-none-any.whl size=107097 sha256=564a8f8126de510dc7cb44407b867cfd4ca62a11085c95255994d564825851d0
  Stored in directory: /Users/przivic/Library/Caches/pip/wheels/11/73/9a/f91ac1f1816436b16423617c5be5db048697ff152a9c4346f2
Successfully built smart-open
You should consider upgrading via the '/Users/przivic/anaconda3/bin/python -m pip install --upgrade pip' command.


In [2]:
from pathlib import Path
import pandas as pd

In [3]:
PATH = Path('../../data/')

In [4]:
import sys
sys.path.append('/Users/przivic/prog/machine_learning_practico')

In [5]:
from lib import data, transformers
from lib.model import get_features_pipe, get_model_pipe

In [30]:
from tqdm.notebook import tqdm
import csv

In [51]:
from gensim.models.callbacks import CallbackAny2Vec
from itertools import groupby

class IterSentences:
    def __init__(self, data_path, some=False):
        self.data_path = data_path
        self.some = some
        
    def __iter__(self):
        reader = csv.DictReader((self.data_path / 'title.principals.tsv').open(), delimiter='\t')
        for i, (tconst, rows) in enumerate(groupby(tqdm(reader), lambda x: x['tconst'])):
            if self.some and i == 1000: break
            yield [e['nconst'] for e in sorted(rows, key=lambda x: int(x['ordering']))]
            

class EpochSaver(CallbackAny2Vec):
    def __init__(self, data_path, fname_prefix=''):
        self.data_path = data_path
        self.fname_prefix = fname_prefix
        self.epoch = 1
    
    def on_epoch_end(self, model):
        print(f'Finished epoch {self.epoch}. Saving...')
        prefix = f'{self.fname_prefix}_' if self.fname_prefix else ''
        
        output_path = self.data_path / 'w2v' / f'{prefix}epoch_{self.epoch}'
        output_path.parent.mkdir(parents=True, exist_ok=True)
        
        model.save(str(output_path))
        self.epoch += 1

In [31]:
from gensim.models import Word2Vec

In [56]:
model = Word2Vec(window=10, iter=10,  callbacks=[EpochSaver(PATH)])

In [57]:
model.build_vocab(IterSentences(PATH))

In [58]:
model.train(IterSentences(PATH), total_words=model.corpus_total_words, epochs=10)


Finished epoch 1. Saving...



Finished epoch 2. Saving...



Finished epoch 3. Saving...



Finished epoch 4. Saving...



Finished epoch 5. Saving...


...


Finished epoch 7. Saving...



Finished epoch 8. Saving...



Finished epoch 9. Saving...



Finished epoch 10. Saving...


(362470430, 410851810)